# <center> Турнир между виртуальными агентами в игре «камень-ножницы-бумага»

1. Описать поведение бота, который будет играть с такими же ботами в игру «камень-ножницы-бумага». Поведение бота описывается с помощью функции, которая принимает на вход информацию о прошлых играх.
2. После описания поведения агентов запустить турнир между ними и проверить, какая стратегия показывает себя лучше всех.

# <center> Подготовка

## Имортируем все необходимые инструмены

In [232]:
import random
from kaggle_environments import evaluate, make

# <center> Создадим агентов с различными стратегиями

## Создадим 8 агентов, каждый с уникальной стратегией выбора хода.

В нашей игре будем придерживаться следующих правил:
<br/> Камень = 0
<br/> Ножницы = 2
<br/> Бумага = 1
<br/> Камень бьёт ножницы, ножницы бьют бумагу, а бумага бьёт камень.

### Агент: Каменный!

Этот агент всегда выбирает камень (0)

In [233]:
def rock_agent(observation, configuration):
    return 0

### Агент: Ножницы навсегда!

Этот агент всегда выбирает ножницы (1)

In [234]:
def scissors_agent(observation, configuration):
    return 2

### Агент: Бумажный!

Этот агент всегда выбирает бумагу (2)

In [235]:
def paper_agent(observation, configuration):
    return 1

### Агент: Случайный

Этот агент выбирает ход случайным образом из возможных вариантов.

In [236]:
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])

### Агент: Упорядоченный

Этот агент чередует свои ходы по порядку: камень → бумага → ножницы → камень и т.д.

In [237]:
def alternating_agent_factory():
    state = {'last_move': -1}
    
    def alternating_agent(observation, configuration):
        state['last_move'] = (state['last_move'] + 1) % 3
        return state['last_move']
    
    return alternating_agent

alternating_agent = alternating_agent_factory()

### Агент: Гюнтер о'Дим (по прозвищу зеркало)

Этот агент повторяет последний ход противника. Если это первая игра, выбирает случайный ход.

In [238]:
def mirror_agent(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])
    return observation.lastOpponentAction

### Агент: Анти-Зеркало

Этот агент выбирает ход, который побеждает последний ход противника. Если это первая игра, выбирает случайный ход.

In [239]:
def anti_mirror_agent(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])
    last_opponent_move = observation.lastOpponentAction
    winning_move = (last_opponent_move + 1) % 3
    return winning_move

### Агент: Камень с приветом!

Этот агент выбирает ход случайным образом, но вероятность камня будет выше:

<br/> Камень (0) — 60%
<br/> Ножницы (2) — 20%
<br/> Бумага (1) — 20%

In [240]:
def rock_random_agent(observation, configuration):
    return random.choices(
        [0, 1, 2],
        weights=[0.6, 0.2, 0.2],
        k=1
    )[0]

### Агент: Ножницы с настроением!

Этот агент выбирает ход случайным образом, но вероятность ножниц будет выше:

<br/> Камень (0) — 20%
<br/> Ножницы (2) — 60%
<br/> Бумага (1) — 20%

In [241]:
def scissors_random_agent(observation, configuration):
    return random.choices(
        [0, 1, 2],
        weights=[0.2, 0.6, 0.2],
        k=1
    )[0]

### Агент: Случайных ход, но предпочитает бумагу

Этот агент выбирает ход случайным образом, но вероятность бумаги будет выше:

<br/> Камень (0) — 20%
<br/> Ножницы (2) — 20%
<br/> Бумага (1) — 60%

In [242]:
def paper_random_agent(observation, configuration):
    return random.choices(
        [0, 1, 2],
        weights=[0.2, 0.2, 0.6],
        k=1
    )[0]

### Агент: Каменные ножницы

Этот агент будет чередовать ножницы и камень

In [243]:
def stone_scissors(observation, configuration):
    if observation.step % 2 == 0:
        return 0
    else:
        return 2

### Агент: Бумажные ножницы

Этот агент будет чередовать ножницы и бумагу

In [244]:
def paper_scissors(observation, configuration):
    if observation.step % 2 == 0:
        return 1
    else:
        return 2

### Агент: Бумажный камень

Этот агент будет чередовать камень и бумагу

In [245]:
def paper_stone(observation, configuration):
    if observation.step % 2 == 0:
        return 0
    else:
        return 1

# <center> Подготовка турнира

### Вначале создадим среду для проведения турнира

In [246]:
env = make("rps", configuration={"episodeSteps": 100}, debug=True)

### Теперь подготовим список участников

In [247]:
agents = {
    "Каменный!": rock_agent,
    "Ножницы навсегда!": scissors_agent,
    "Бумажный!": paper_agent,
    "Случайный": random_agent,
    "Упорядоченный": alternating_agent,
    "Гюнтер о'Дим": mirror_agent,
    "Анти-Зеркало": anti_mirror_agent,
    "Камень с приветом!": rock_random_agent,
    "Ножницы с настроением!": scissors_random_agent,
    "Бумага на выбор!": paper_random_agent,
    "Каменные ножницы": stone_scissors,
    "Бумажные ножницы": paper_scissors,
    "Бумажный камень": paper_stone
}

### Проведение турнира между всеми агентами

In [248]:
def run_tournament(agent1, agent2, num_episodes):
    result = evaluate("rps", [agent1, agent2], num_episodes=num_episodes)
    return result

### Установим колличество раундов между каждой из пар на 10

In [249]:
num_episodes = 10

### Сделаем словарь где будут хранится результаты игры

In [250]:
results = {name: {"wins": 0, "losses": 0} for name in agents.keys()}

# <center> Проведение турнира!

### Запуск турнира

In [251]:
for i, agent1_name in enumerate(agents.keys()):
    for j, agent2_name in enumerate(list(agents.keys())[i + 1:], start=i + 1):
        # Получаем результаты
        result = run_tournament(agents[agent1_name], agents[agent2_name], num_episodes)

        # Подсчет побед
        wins1 = sum(1 for game_result in result if game_result[0] > game_result[1])
        wins2 = sum(1 for game_result in result if game_result[1] > game_result[0])

        # Обновляем результаты
        results[agent1_name]["wins"] += wins1
        results[agent2_name]["wins"] += wins2
        results[agent1_name]["losses"] += wins2
        results[agent2_name]["losses"] += wins1

# <center> Результаты

### Выведем результаты и узнаем кто победил

In [252]:
print("\nРезультаты турнира:")
print("Агент | Победы | Поражения | % Побед")
print("-" * 40)
for agent_name, result in results.items():
    total_wins = result["wins"]
    total_losses = result["losses"]
    total_matches = total_wins + total_losses
    win_percentage = (total_wins / total_matches * 100) if total_matches > 0 else 0
    print(f"{agent_name:<20} | {total_wins:<6} | {total_losses:<9} | {win_percentage:.2f}%")


Результаты турнира:
Агент | Победы | Поражения | % Побед
----------------------------------------
Каменный!            | 34     | 42        | 44.74%
Ножницы навсегда!    | 36     | 45        | 44.44%
Бумажный!            | 33     | 47        | 41.25%
Случайный            | 20     | 34        | 37.04%
Упорядоченный        | 23     | 5         | 82.14%
Гюнтер о'Дим         | 12     | 24        | 33.33%
Анти-Зеркало         | 67     | 35        | 65.69%
Камень с приветом!   | 38     | 51        | 42.70%
Ножницы с настроением! | 39     | 56        | 41.05%
Бумага на выбор!     | 41     | 51        | 44.57%
Каменные ножницы     | 48     | 33        | 59.26%
Бумажные ножницы     | 56     | 22        | 71.79%
Бумажный камень      | 40     | 42        | 48.78%


# <center> Выводы

Мы поэтапно разработали систему для игры "Камень-Ножницы-Бумага", включающую разнообразные стратегии агентов. Проведя турнир между этими агентами, мы определили наиболее эффективную стратегию. На основании полученных результатов выяснили, что стратегия "Упорядоченный" показала лучшие результаты. Однако, учитывая, что количество игр между претендентами составило всего 10, можно сделать вывод о том, что данный результат может быть недостаточно надежным. Для более точного анализа и оценки эффективности стратегий целесообразно провести большее количество игр между агентами.